In [2]:
#imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

In [6]:
#load data
qb24 = pd.read_csv('data/QB24.csv')
qb23 = pd.read_csv('data/QB23.csv')
qb22 = pd.read_csv('data/QB22.csv')

rb24 = pd.read_csv('data/RB24.csv')
rb23 = pd.read_csv('data/RB23.csv')
rb22 = pd.read_csv('data/RB22.csv')

wr24 = pd.read_csv('data/WR24.csv')
wr23 = pd.read_csv('data/WR23.csv')
wr22 = pd.read_csv('data/WR22.csv')

te24 = pd.read_csv('data/TE24.csv')
te23 = pd.read_csv('data/TE23.csv')
te22 = pd.read_csv('data/TE22.csv')

k24 = pd.read_csv('data/K24.csv')
k23 = pd.read_csv('data/K23.csv')
k22 = pd.read_csv('data/K22.csv')

dst24 = pd.read_csv('data/DST24.csv')
dst23 = pd.read_csv('data/DST23.csv')
dst22 = pd.read_csv('data/DST22.csv')

Data Visualization

In [8]:
qb24.head()

,Rank,Player,CMP,ATT,PCT,YDS,Y/A,TD,INT,SACKS,ATT.1,YDS.1,TD.1,FL,G,FPTS,FPTS/G,ROST
0,1.0,Lamar Jackson (BAL),316.0,474.0,66.7,"4,172",8.8,41.0,4.0,23.0,139.0,915.0,4.0,5.0,17.0,434.4,25.6,98.9%
1,2.0,Josh Allen (BUF),307.0,483.0,63.6,"3,731",7.7,28.0,6.0,14.0,102.0,531.0,12.0,2.0,17.0,385.1,22.7,99.2%
2,3.0,Joe Burrow (CIN),460.0,652.0,70.6,"4,918",7.5,43.0,9.0,48.0,42.0,201.0,2.0,5.0,17.0,381.9,22.5,95.3%
3,4.0,Baker Mayfield (TB),407.0,570.0,71.4,"4,500",7.9,41.0,16.0,40.0,60.0,378.0,3.0,2.0,17.0,381.8,22.5,89.8%
4,5.0,Jayden Daniels (WAS),331.0,480.0,69.0,"3,568",7.4,25.0,9.0,47.0,148.0,891.0,6.0,0.0,17.0,364.7,21.5,96.5%


In [10]:
rb24.head()

,Rank,Player,ATT,YDS,Y/A,LG,20+,TD,REC,TGT,YDS.1,Y/R,TD.1,FL,G,FPTS,FPTS/G,ROST
0,1.0,Saquon Barkley (PHI),345.0,"2,005",5.8,72.0,17.0,13.0,33.0,43.0,278.0,8.4,2.0,1.0,16.0,322.3,20.1,99.1%
1,2.0,Derrick Henry (BAL),325.0,"1,921",5.9,87.0,19.0,16.0,19.0,22.0,193.0,10.2,2.0,1.0,17.0,317.4,18.7,98.9%
2,3.0,Jahmyr Gibbs (DET),250.0,"1,412",5.6,70.0,13.0,16.0,52.0,63.0,517.0,9.9,4.0,1.0,17.0,310.9,18.3,98.9%
3,4.0,Bijan Robinson (ATL),304.0,"1,456",4.8,37.0,5.0,14.0,61.0,72.0,431.0,7.1,1.0,0.0,17.0,280.7,16.5,98.0%
4,5.0,Josh Jacobs (GB),301.0,"1,329",4.4,38.0,7.0,15.0,36.0,43.0,342.0,9.5,1.0,3.0,17.0,257.1,15.1,98.4%


In [11]:
te24.head()

,Rank,Player,REC,TGT,YDS,Y/R,LG,20+,TD,ATT,YDS.1,TD.1,FL,G,FPTS,FPTS/G,ROST
0,1.0,George Kittle (SF),78.0,94.0,"1,106",14.2,43.0,21.0,8.0,0.0,0.0,0.0,0.0,15.0,158.6,10.6,95.3%
1,2.0,Brock Bowers (LV),112.0,153.0,"1,194",10.7,57.0,15.0,5.0,5.0,13.0,0.0,0.0,17.0,150.7,8.9,98.3%
2,3.0,Trey McBride (ARI),111.0,147.0,"1,146",10.3,37.0,10.0,2.0,1.0,2.0,1.0,0.0,16.0,138.8,8.7,95.3%
3,4.0,Jonnu Smith (MIA),88.0,111.0,884,10.0,57.0,4.0,8.0,2.0,-1.0,0.0,1.0,17.0,134.3,7.9,83.8%
4,5.0,Mark Andrews (BAL),55.0,69.0,673,12.2,67.0,6.0,11.0,4.0,5.0,0.0,0.0,17.0,133.8,7.9,90.4%


In [12]:
wr24.head()

,Rank,Player,REC,TGT,YDS,Y/R,LG,20+,TD,ATT,YDS.1,TD.1,FL,G,FPTS,FPTS/G,ROST
0,1.0,Ja'Marr Chase (CIN),127.0,175.0,"1,708",13.4,70.0,19.0,17.0,3.0,32.0,0.0,0.0,17.0,276.0,16.2,98.7%
1,2.0,Justin Jefferson (MIN),103.0,154.0,"1,533",14.9,97.0,28.0,10.0,1.0,3.0,0.0,0.0,17.0,214.5,12.6,99.2%
2,3.0,Amon-Ra St. Brown (DET),115.0,141.0,"1,263",11.0,66.0,14.0,12.0,2.0,6.0,0.0,1.0,17.0,201.2,11.8,98.9%
3,4.0,Brian Thomas Jr. (JAC),87.0,133.0,"1,282",14.7,85.0,18.0,10.0,6.0,48.0,0.0,0.0,17.0,197.0,11.6,97.7%
4,5.0,Terry McLaurin (WAS),82.0,117.0,"1,096",13.4,86.0,12.0,13.0,2.0,2.0,0.0,1.0,17.0,185.8,10.9,95.3%


In [13]:
k24.head()

,Rank,Player,FG,FGA,PCT,LG,1-19,20-29,30-39,40-49,50+,XPT,XPA,G,FPTS,FPTS/G,ROST
0,1.0,Brandon Aubrey (DAL),40.0,47.0,85.1,65.0,0.0,4.0,8.0,14.0,14.0,30.0,30.0,17.0,192.0,11.3,87.9%
1,2.0,Chris Boswell (PIT),41.0,44.0,93.2,57.0,0.0,9.0,12.0,7.0,13.0,35.0,35.0,17.0,191.0,11.2,55.3%
2,3.0,Cameron Dicker (LAC),39.0,42.0,92.9,59.0,1.0,9.0,9.0,11.0,9.0,33.0,36.0,17.0,179.0,10.5,82.6%
3,4.0,Ka'imi Fairbairn (HOU),36.0,42.0,85.7,59.0,0.0,8.0,11.0,4.0,13.0,34.0,36.0,17.0,172.0,10.1,47.2%
4,5.0,Jason Sanders (MIA),37.0,41.0,90.2,57.0,0.0,7.0,13.0,5.0,12.0,26.0,28.0,17.0,166.0,9.8,42.4%


In [14]:
dst24.head()

,Rank,Player,SACK,INT,FR,FF,DEF TD,SFTY,SPC TD,G,FPTS,FPTS/G,ROST
0,1.0,Denver Broncos (DEN),63.0,15.0,9.0,12.0,5.0,2.0,0.0,17.0,147.0,8.6,83.1%
1,2.0,Minnesota Vikings (MIN),49.0,24.0,9.0,10.0,3.0,0.0,0.0,17.0,125.0,7.4,74.3%
2,3.0,Chicago Bears (CHI),40.0,11.0,13.0,14.0,1.0,0.0,2.0,17.0,121.0,7.1,24.5%
3,4.0,Houston Texans (HOU),49.0,19.0,9.0,10.0,3.0,1.0,0.0,17.0,117.0,6.9,43.7%
4,5.0,Seattle Seahawks (SEA),45.0,13.0,5.0,14.0,4.0,1.0,1.0,17.0,113.0,6.6,27.0%
